Stage 1: Data collection
- Create CV file where any stock entered data is automatically parsed and saved

In [18]:
import csv
import yfinance as yf

# Step 1: Create or locate the CSV file
csv_file = "StockData.csv"

try:
    # Check if the file already exists
    with open(csv_file, "r"):
        pass
except FileNotFoundError:
    # Create a new file with header if it doesn't exist
    with open(csv_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Ticker Symbol", "Average volume", "t-13 vol", "t-13 change", "t-12 vol", "t-12 change", "t-11 vol", "t-11 change", "t-10 vol", "t-10 change", "t-9 vol", "t-9 change", "t-8 vol", "t-8 change", "t-7 vol", "t-7 change","t-6 vol", "t-6 change","t-5 vol", "t-5 change","t-4 vol", "t-4 change", "t-3 vol",
                         "t-3 change", "t-2 volume",
                         "t-2 change", "t-1 volume",
                         "t-1 change", "Result change", "Bucket"])

# Step 2: Take input of ticker symbol from user
ticker = input("Enter the ticker symbol: ")

# Step 3: Use Yahoo Finance API to search for the ticker
try:
    stock = yf.Ticker(ticker)
    info = stock.history(period="14d")
    #print(info)

    # Step 4: Return error if ticker does not exist
    if info.empty:
        print("Ticker does not exist.")
    else:
        # Step 5: Create a new row in the CSV file
        with open(csv_file, "a", newline="") as file:
            writer = csv.writer(file)
            avgVolume = info["Volume"].mean()
            # Step 6: Add information to the row
            row = [
                ticker,
                info["Volume"].mean(),  # Average volume

                info["Volume"].iloc[0]/avgVolume,  # 13 days ago's volume
                (info["Close"].iloc[0] - info["Open"].iloc[0])/(info["Open"].iloc[0])*100,  # 13 days ago's daily % change

                info["Volume"].iloc[1]/avgVolume,  # 12 days ago's volume
                (info["Close"].iloc[1] - info["Open"].iloc[1])/(info["Open"].iloc[1])*100,  # 12 days ago's daily % change

                info["Volume"].iloc[2]/avgVolume,  # 11 day ago's volume
                (info["Close"].iloc[2] - info["Open"].iloc[2])/(info["Open"].iloc[2])*100,  # 11 day ago's daily % change

                info["Volume"].iloc[3]/avgVolume,  # 10 day ago's volume
                (info["Close"].iloc[3] - info["Open"].iloc[3])/(info["Open"].iloc[3])*100,  # 10 day ago's daily % change

                info["Volume"].iloc[4]/avgVolume,  
                (info["Close"].iloc[4] - info["Open"].iloc[4])/(info["Open"].iloc[4])*100, 

                info["Volume"].iloc[5]/avgVolume,  
                (info["Close"].iloc[5] - info["Open"].iloc[5])/(info["Open"].iloc[5])*100, 
                
                info["Volume"].iloc[6]/avgVolume,  
                (info["Close"].iloc[6] - info["Open"].iloc[6])/(info["Open"].iloc[6])*100, 

                info["Volume"].iloc[7]/avgVolume,  
                (info["Close"].iloc[7] - info["Open"].iloc[7])/(info["Open"].iloc[7])*100, 

                info["Volume"].iloc[8]/avgVolume,  
                (info["Close"].iloc[8] - info["Open"].iloc[8])/(info["Open"].iloc[8])*100, 

                info["Volume"].iloc[9]/avgVolume,  
                (info["Close"].iloc[9] - info["Open"].iloc[9])/(info["Open"].iloc[9])*100,

                info["Volume"].iloc[10]/avgVolume,  
                (info["Close"].iloc[10] - info["Open"].iloc[10])/(info["Open"].iloc[10])*100,

                info["Volume"].iloc[11]/avgVolume,  
                (info["Close"].iloc[11] - info["Open"].iloc[11])/(info["Open"].iloc[11])*100,

                info["Volume"].iloc[12]/avgVolume,  
                (info["Close"].iloc[12] - info["Open"].iloc[12])/(info["Open"].iloc[4])*100,    

               (info["Close"].iloc[13] - info["Open"].iloc[13])/(info["Open"].iloc[13])*100,  # 0 days ago daily %
            ]
            writer.writerow(row)
            print("Data saved successfully.")

except Exception as e:
    print("An error occurred:", str(e))


Data saved successfully.


Ml part
- Create buckets of % change
- Create SVM model

In [22]:
#Create buckets

import pandas as pd
df = pd.read_csv(csv_file)

def makeBins(s): #s is the result change
    if (s < 0): #negative change
        return 0
    if(s > 0): #positive change
        return 1
    
df['BIN'] = df['Result change'].apply(makeBins, 0)
df

,Ticker Symbol,Average volume,t-13 vol,t-13 change,t-12 vol,t-12 change,t-11 vol,t-11 change,t-10 vol,t-10 change,...,t-4 vol,t-4 change,t-3 vol,t-3 change,t-2 volume,t-2 change,t-1 volume,t-1 change,Result change,BIN
0,edn,6.015000e+04,1.579385,14.931232,0.816293,2.649576,0.663342,-1.850297,0.310889,-0.841754,...,0.842893,2.568487,1.795511,11.811024,1.810474,-0.071430,3.334996,-5.448446,6.159416,1
1,sien,3.926343e+06,0.011716,-5.074622,0.005756,1.886791,0.010137,4.968947,0.793079,8.695651,...,0.905626,25.543473,0.080278,-3.493447,0.086773,4.017853,0.039528,-1.307188,14.864861,1
2,izm,1.069357e+05,1.409258,1.574802,0.970677,1.550392,1.153964,2.879585,1.300782,12.781955,...,2.112484,28.301892,0.513393,-2.631581,0.530225,11.945093,0.610647,3.695654,8.196727,1
3,lyra,2.296571e+05,1.648109,-5.956115,0.673177,-1.999998,0.342685,1.742167,0.239052,-2.372879,...,0.262565,1.337791,3.776499,17.142855,1.024571,5.027935,1.243593,-1.374569,16.085788,1
4,lmdx,2.134100e+06,0.041235,-2.325586,0.065273,2.439022,0.019024,-2.272725,0.060962,-1.999994,...,0.022586,0.273969,1.888993,-4.109593,10.030645,-3.063457,0.934586,13.043479,8.510637,1
